In [1]:
import redis
import json
from functools import partial
import numpy as np
# IMPORTANT: Run redis-server in a shell window for it to work.
stateDB = redis.Redis(host='localhost', port=6379, db=0)
# Just making a default entry into database for state for epsilon greedy algorithm
rewards = np.zeros((3,))
stateDB.hset("state1", "rewards", str(rewards)[1:-1])
stateDB.hset("state1", 'epsilon', 0.5)
def selection_policy(state):
    epsilon = load_var(state, 'epsilon', float)
    print('Epsilon:', epsilon)
    rewards = load_var(state, 'rewards', partial(np.fromstring, sep=' '))
#     return 0
    if np.random.rand() < epsilon:
        return np.random.randint(0, 3)
    else:
        return np.argmax(rewards)
def gen_res(model_id, inp):
    return {model_id: model_id * inp + 1}
def combine(state, stateDB, results, dbName):
    epsilon = load_var(state, 'epsilon', float)
    rewards = load_var(state, 'rewards', partial(np.fromstring, sep=' '))
    for model, result in results.items():
        rewards[model] = result
    print('Rewards:', rewards)
    epsilon = 0.9*epsilon
    stateDB.hset(dbName, 'epsilon', epsilon)
    stateDB.hset(dbName, 'rewards', str(rewards)[1:-1])
    return max(results.values())
# Helper function to deal with redis converting everything to byte strings
def load_var(state, var_name, type_cast_func):
    return type_cast_func(state[var_name.encode('utf-8')])
counts = np.zeros((3,))
def qf(query):
    q = json.loads(query)['input'] # Will be a list of stuff
    state = stateDB.hgetall('state1') # Will be a dic of state variables
    model_id = selection_policy(state)
    print("Models Selected", model_id)
    counts[model_id] += 1
    results = gen_res(model_id, q)
    return combine(state, stateDB, results, 'state1')

In [2]:
for i in range(30):
    q = json.dumps({'input': i})
    print(qf(q))
print(counts / 30)

Epsilon: 0.5
Models Selected 2
Rewards: [ 0.  0.  1.]
1
Epsilon: 0.45
Models Selected 2
Rewards: [ 0.  0.  3.]
3
Epsilon: 0.405
Models Selected 2
Rewards: [ 0.  0.  5.]
5
Epsilon: 0.36450000000000005
Models Selected 1
Rewards: [ 0.  4.  5.]
4
Epsilon: 0.32805000000000006
Models Selected 2
Rewards: [ 0.  4.  9.]
9
Epsilon: 0.2952450000000001
Models Selected 2
Rewards: [  0.   4.  11.]
11
Epsilon: 0.2657205000000001
Models Selected 2
Rewards: [  0.   4.  13.]
13
Epsilon: 0.23914845000000007
Models Selected 2
Rewards: [  0.   4.  15.]
15
Epsilon: 0.21523360500000008
Models Selected 2
Rewards: [  0.   4.  17.]
17
Epsilon: 0.19371024450000007
Models Selected 2
Rewards: [  0.   4.  19.]
19
Epsilon: 0.17433922005000008
Models Selected 2
Rewards: [  0.   4.  21.]
21
Epsilon: 0.15690529804500009
Models Selected 1
Rewards: [  0.  12.  21.]
12
Epsilon: 0.14121476824050008
Models Selected 1
Rewards: [  0.  13.  21.]
13
Epsilon: 0.12709329141645007
Models Selected 2
Rewards: [  0.  13.  27.]
27
Eps